In [ ]:
import numpy as np
import math
import cmath as cm
import matplotlib.pyplot as plt
import jax.numpy as jnp
from jax import jacfwd, random

# from sklearn.preprocessing import StandardScaler

In [ ]:
def split_array(arr, index):
    return [arr[:index], arr[index:]]

data = []
for ch in range(4):
    for H in range(3):
        with open(f"Calibration Example_915/ch{ch + 1}_H{H + 1}.txt") as file:
            for line in file:
                try:
                    row = [float(num) for num in line.split()]
                    data.append([H, ch] + row)
                except:
                    pass

# def create_un_mat(v):
#     Mc = np.zeros((4, 4),dtype=np.complex128)
#     Mc[0][1] = v[0] + 1j * v[1]
#     Mc[1][0] = v[0] - 1j * v[1]
    
#     Mc[0][2] = v[2] + 1j * v[3]
#     Mc[2][0] = v[2] - 1j * v[3]
    
#     Mc[0][3] = v[4] + 1j * v[5]
#     Mc[3][0] = v[4] - 1j * v[5]

#     Mc[1][2] = v[6] + 1j * v[7]
#     Mc[2][1] = v[6] - 1j * v[7]

    # Mc[1][3] = v[8] + 1j * v[9]
    # Mc[3][1] = v[8] - 1j * v[9]

    # Mc[2][3] = v[10] + 1j * v[11]
    # Mc[3][2] = v[10] - 1j * v[11]
    
    # U = expm(1j * Mc)
    
    # return U

def T_pq(N, p, q, phi, theta):
    T = jnp.eye(N,dtype=jnp.complex128)
    T=T.at[q,q].set(jnp.exp(1j*phi)*jnp.sin(theta)*1j*jnp.exp(1j*theta*0.5)) #with coefficient
    T=T.at[q,p].set(jnp.exp(1j*phi)*jnp.cos(theta)*1j*jnp.exp(1j*theta*0.5))
    T=T.at[p,q].set(jnp.cos(theta)*1j*jnp.exp(1j*theta*0.5))
    T=T.at[p,p].set(-jnp.sin(theta)*1j*jnp.exp(1j*theta*0.5))

    #  x = x.at[idx].set(y)
    return T

def create_un_mat(N, v):
    k=0
    key = random.PRNGKey(0)
    phases_last= random.normal(key, shape=(3,)) #np.zeros(3)
    phases1, phases2 = jnp.split(jnp.concatenate((v,random.normal(key, shape=(3,)) )),2)
    phases = jnp.hstack((phases1.reshape(-1, 1), phases2.reshape(-1, 1)))

    U=jnp.eye(N, dtype=jnp.complex128)
    for p in range(1, N):
        for q in range(0, p):
            U=U.dot(T_pq(N, p, q, phases[k, 0], phases[k, 1])) #be aware of precise phase sequence!!!
            k+=1
    D=jnp.eye(N, dtype=jnp.complex128)
    for i in range(N-1):
        D=D.at[i,i].set(cm.exp(1j*phases_last[i]))
    U=U.dot(D)
#    return U
    return jnp.linalg.inv(U)



def f(v, data):
    M1 = create_un_mat(v[:12])
    M2 = create_un_mat(v[12:24])
    alpha = np.reshape(v[24:33], (3, 3))
    h_0 = v[33:36]
    cost = 0
    for i in data:
        x = np.zeros(3)
        x[i[0]] = 10**(-5) * i[2] ** 2
        y = i[3:]
        y /= np.sum(y)
        h_list = h_0 + alpha.dot(np.transpose(x))
        H = np.diag([math.e ** (1j * h) for h in h_list] + [1])
        predict = (np.abs(np.transpose(M2.dot(H).dot(M1)))** 2)[i[1]]
        cost += np.linalg.norm(np.array(predict) - y)
    return cost


def f_nd(v, data=data):
    # global data
    M1 = create_un_mat(4, v[:9])
    M2 = create_un_mat(4, v[9:18])
    alpha = jnp.reshape(4, v[18:27], (3, 3))
    h_0 = v[27:30]
    cost = 0
    for i in data:
        x = jnp.zeros(3)
        x[i[0]] = 10**(-5) * i[2] ** 2
        y = i[3:]
        y /= jnp.sum(y)
        h_list = h_0 + alpha.dot(jnp.transpose(x))
        H = jnp.diag([math.e ** (1j * h) for h in h_list] + [1])
        predict = (jnp.abs(jnp.transpose(M2.dot(H).dot(M1)))** 2)[i[1]]
        cost += jnp.linalg.norm(jnp.array(predict) - y)
    return cost


def numerical_gradient(data, x, epsilon=1e-8):
    grad = np.zeros_like(x)
    f_0 = f(x, data)
    for i in range(len(x)):
        x_up = np.copy(x)
        x_down = np.copy(x)
        x_up[i] += epsilon
        x_down[i] -= epsilon
        grad[i] = (f_0 - f(x_down, data)) / (2 * epsilon)
    return np.sign(grad)

def gradient_descent(data, x0, learning_rate_start=0.01, learning_rate_finish=0.002, max_iter=1000, tol=1e-4):
    x = np.array(x0, dtype=float)
    dt_vec = np.ones(len(x0)) * learning_rate_start
    for i in range(max_iter):
        dt_vec = np.clip(dt_vec, 0, learning_rate_start)
        f0 = f(x, data)
        for j in range(len(x0)):
            x[j] += dt_vec[j]
            f_up = f(x, data)
            x[j] -= 2 * dt_vec[j]
            f_down = f(x, data)
            if f_up < f0:
                x[j] += 2 * dt_vec[j]
            elif f_down > f0:
                x[j] += dt_vec[j]
                dt_vec[j] *= 0.8
            dt_vec[j] *= 1.03
        if i % 10 == 0:
            print(x)
            print("Iteration = ", i)
            print(f(x, data))
            print(dt_vec)
    return x, f(x, data)

def gradient_descent_nd(data, x0, learning_rate_start=0.01, learning_rate_finish=0.002, max_iter=1000, tol=1e-4):
    x = np.array(x0, dtype=float)
    dt_vec = np.ones(len(x0)) * learning_rate_start
    for i in range(max_iter):
        dt_vec = np.clip(dt_vec, 0, learning_rate_start)
        f0 = f_nd(x, data)
        for j in range(len(x0)):
            x[j] += dt_vec[j]
            f_up = f_nd(x, data)
            x[j] -= 2 * dt_vec[j]
            f_down = f_nd(x, data)
            if f_up < f0:
                x[j] += 2 * dt_vec[j]
            elif f_down > f0:
                x[j] += dt_vec[j]
                dt_vec[j] *= 0.8
            dt_vec[j] *= 1.03
        if i % 10 == 0:
            print(x)
            print("Iteration = ", i)
            print(f_nd(x, data))
            print(dt_vec)
    return x, f(x, data)

In [ ]:
# def f(x):
#     return jnp.array([x[0]**2 + x[1], jnp.sin(x[0]) * x[1]])

J = jacfwd(f_nd)(jnp.array([1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0]))
print(J)  # Shape: (2, 2)

In [ ]:
# x0 = [-0.41787547, -0.18504808,  0.44831782, -0.9248506 ,  0.4016049 , -0.24746775,
#  -0.04578627,  1.3592343 , -0.67259892,  0.83798169,  0.37996106,  0.25830816,
#   0.12016575,  1.94091927,  0.92471875,  0.4298926 ,  0.78182305,  1.48576622,
#   0.28119859,  0.90557274,  0.28573387,  0.93179326,  1.37962862,  0.32020062,
#   2.44455675, -0.21100629,  0.28743759, -0.17984061,  0.03300196,  0.89548313,
#   0.12244837,  0.63345639,  0.23770722, -0.26129656,  0.40731544, -1.36835074]
x0 = np.random.random(36)
# x0 = minimum_x_iter

result_minimize = basinhopping(f, x0, niter = 2)
# result_minimize = minimize(f_nd, x0)

print(result_minimize.fun)
# print(create_un_mat(x0[0:12]))

In [ ]:
# x0 = [-0.41787547, -0.18504808,  0.44831782, -0.9248506 ,  0.4016049 , -0.24746775,
#  -0.04578627,  1.3592343 , -0.67259892,  0.83798169,  0.37996106,  0.25830816,
#   0.12016575,  1.94091927,  0.92471875,  0.4298926 ,  0.78182305,  1.48576622,
#   0.28119859,  0.90557274,  0.28573387,  0.93179326,  1.37962862,  0.32020062,
#   2.44455675, -0.21100629,  0.28743759, -0.17984061,  0.03300196,  0.89548313,
#   0.12244837,  0.63345639,  0.23770722, -0.26129656,  0.40731544, -1.36835074]
x0 = np.random.random(36)
# x0 = minimum_x_iter

result_minimize = basinhopping(f, x0, niter = 2)
# result_minimize = minimize(f_nd, x0)

print(result_minimize.fun)
# print(create_un_mat(x0[0:12]))

In [ ]:
print(result_minimize.x)
print(f_nd(result_minimize.x))
# print(len(data))
# minimum_x = result.x

In [ ]:
minimum_f_iter = 1000
minimum_x_iter = np.ones(36)
for i in range(100):
    x0=np.random.random(36)
    minimum_x, minimum_f = gradient_descent_nd(data, x0, learning_rate_start=0.05, learning_rate_finish=0.002, max_iter=50)
    if minimum_f<minimum_f_iter:
        minimum_f_iter = minimum_f
        minimum_x_iter = minimum_x

print(f"Минимум функции: f({minimum_x}) = {minimum_f}")

In [ ]:
minimum_f_iter

In [ ]:
# params = result_minimize.x
params = [ 0.33266165, -0.01319983 , 1.08954383 , 1.07561339, -0.23657239 , 0.41973725,
  1.2090425,   0.47662948,  1.26401492,  0.56776001,  0.31562289,  0.83814405,
  0.28682899,  0.2724307 ,  0.21488137,  0.71553312,  0.86192567,  0.68798451,
  0.15807603,  0.62079314,  0.99481647,  0.24612045,  0.28329745,  0.65160988,
  1.60921787,  0.95705642,  0.92694837,  0.85362065, -0.29354937,  1.45062795,]

# M1 = create_un_mat(params[:12])
# M2 = create_un_mat(params[12:24])
# alpha = np.reshape(params[24:33], (3, 3))
# h_0 = params[33:36]
M1 = create_un_mat(4, params[:9])
M2 = create_un_mat(4, params[9:18])
alpha = np.reshape(params[18:27], (3, 3))
h_0 = params[27:30]

fig, ax = plt.subplots(3, 4, figsize=(16, 10))

for j in range(12):

    for i in data[131*j:131*(j+1)]:
        x = np.zeros(3)
        x[i[0]] = 10**(-5)*i[2] ** 2
        h_list = h_0 + alpha.dot(np.transpose(x))
        H = np.diag([math.e ** (1j * h) for h in h_list] + [1])
        result = (np.abs(np.transpose(M2.dot(H).dot(M1))) ** 2)[i[1]]
        ax[j//4][j%4].scatter(i[2], result[0], color = 'blue')
        ax[j//4][j%4].scatter(i[2], result[1], color = 'green')
        ax[j//4][j%4].scatter(i[2], result[2], color = 'yellow')
        ax[j//4][j%4].scatter(i[2], result[3], color = 'red')

    ax[j//4][j%4].plot([d[2] for d in data[131*j:131*(j+1)]], [d[3]/sum(d[3:]) for d in data[131*j:131*(j+1)]])#, color = 'blue')
    ax[j//4][j%4].plot([d[2] for d in data[131*j:131*(j+1)]], [d[4]/sum(d[3:]) for d in data[131*j:131*(j+1)]])#, color = 'green')
    ax[j//4][j%4].plot([d[2] for d in data[131*j:131*(j+1)]], [d[5]/sum(d[3:]) for d in data[131*j:131*(j+1)]])#, color = 'yellow')
    ax[j//4][j%4].plot([d[2] for d in data[131*j:131*(j+1)]], [d[6]/sum(d[3:]) for d in data[131*j:131*(j+1)]])#, color = 'red')

    ax[0][0].set_xlabel('Сила тока, единицы')
    ax[0][0].set_ylabel('Выходное излучение (норм)')

    fig.suptitle('Фитирование данных Ильи с помощью нашего кода')

   
plt.show()

